In [1]:
casa = 'Atlético'
fora = 'Lanús'

## Importando as bibliotecas

In [2]:
import pandas as pd
import numpy as np
import mysql.connector
import os
import matplotlib.pyplot as plt
import seaborn as sns

## Importando as informações do banco de dados

#### Configurações do banco de dados

In [3]:
config = {
    "host": "127.0.0.1",
    "user": "admin",
    "password": "1234",
    "database": "bet_dados"
}

In [4]:
conn = mysql.connector.connect(**config)

#### Query para a tabela "partidas"

In [5]:
query_partidas = f"""
SELECT
    CASE
        WHEN casa = '{casa}' OR fora = '{casa}' THEN '{casa}'
        WHEN casa = '{fora}' OR fora = '{fora}' THEN '{fora}'
        ELSE 'Verificar'
    END AS clube_analise,
    data,
    temporada,
    id_jogo,
    campeonato,
    casa,
    gols_casa,
    gols_fora,
    fora,
    odd_casa,
    odd_empate,
    odd_fora,
    CASE
        WHEN gols_casa > gols_fora THEN casa
            WHEN gols_fora > gols_casa THEN fora
            WHEN gols_casa = gols_fora THEN 'Empate'
    END AS vencedor,
    gols_casa + gols_fora AS total_gols
    FROM
        partidas
    WHERE
        (casa = "{casa}" OR fora = "{casa}" OR casa = "{fora}" OR fora = "{fora}");
"""

#### Query para a tabela "1_tempo"

In [6]:
query_tempo1 = f'''
SELECT
    p.data,
    CASE
        WHEN t1.clube = p.casa THEN 'Casa'
        WHEN t1.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando_certo,
    CASE
        WHEN t1.clube = p.casa THEN p.fora
        WHEN t1.clube = p.fora THEN p.casa
        ELSE 'Verificar'
    END AS adversario,
    p.campeonato,
    t1.*
FROM
    1_tempo t1
    INNER JOIN partidas p ON t1.id_jogo = p.id_jogo
WHERE
    p.casa IN ('{casa}', '{fora}')
    OR p.fora IN ('{casa}', '{fora}');
'''

#### Query para a tabela "2_tempo"

In [7]:
query_tempo2 = f'''
SELECT
    p.data,
    CASE
        WHEN t2.clube = p.casa THEN 'Casa'
        WHEN t2.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando_certo,
    CASE
        WHEN t2.clube = p.casa THEN p.fora
        WHEN t2.clube = p.fora THEN p.casa
        ELSE 'Verificar'
    END AS adversario,
    p.campeonato,
    t2.*
FROM
    2_tempo t2
    INNER JOIN partidas p ON t2.id_jogo = p.id_jogo
WHERE
    p.casa IN ('{casa}', '{fora}')
    OR p.fora IN ('{casa}', '{fora}');
'''

#### Query para a tabela "estatisticas_partidas"

In [8]:
query_est_partidas = f"""
    SELECT
        CASE
            WHEN p.casa = '{casa}' OR p.fora = '{casa}' THEN '{casa}'
            WHEN p.casa = '{fora}' OR p.fora = '{fora}' THEN '{fora}'
            ELSE 'Verificar'
        END AS clube_analise,
        p.data,
        CASE
            WHEN ep.clube = p.casa THEN 'Casa'
            WHEN ep.clube = p.fora THEN 'Fora'
            ELSE 'Verificar'
        END AS mando_certo,
        CASE
            WHEN ep.clube = p.casa THEN p.fora
            WHEN ep.clube = p.fora THEN p.casa
            ELSE 'Verificar'
        END AS adversario,
        p.campeonato,
        ep.*
    FROM
        estatisticas_partidas ep
        INNER JOIN partidas p ON ep.id_jogo = p.id_jogo
    WHERE
        EXISTS (
            SELECT 1
            FROM estatisticas_partidas ep2
            WHERE ep2.id_jogo = ep.id_jogo
              AND ep2.clube IN ('{casa}', '{fora}')
        );
"""

#### Query para a tabela "estatisticas_jogadores"

In [9]:
query_jogadores = f"""
SELECT
    p.data,
    p.id_jogo,
    CASE
        WHEN ej.team = p.casa THEN 'Casa'
        WHEN ej.team = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando_certo,
    CASE
        WHEN ej.team = p.casa THEN p.fora
        WHEN ej.team = p.fora THEN p.casa
        ELSE 'Verificar'
    END AS adversario,
    p.campeonato,
    CASE
        WHEN ej.position = 'G' THEN 'Goleiro'
        WHEN ej.position = 'D' THEN 'Defensor'
        WHEN ej.position = 'M' THEN 'Meio-campo'
        WHEN ej.position = 'F' THEN 'Atacante'
        ELSE 'Verificar'
    END AS position,
    ej.*
FROM
    estatisticas_jogadores ej
    INNER JOIN partidas p ON ej.id_jogo = p.id_jogo
WHERE
    ej.minutes_played > 0
    AND team IN ("{casa}", "{fora}");
"""

#### Criando os DataFrames e encerrando as conexões

Partidas

In [10]:
df_partidas = pd.read_sql(query_partidas, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_34944\2950495610.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_partidas = pd.read_sql(query_partidas, conn)


Primeiro tempo

In [11]:
df_tempo1 = pd.read_sql(query_tempo1, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_34944\2256018260.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tempo1 = pd.read_sql(query_tempo1, conn)


Segundo tempo

In [12]:
df_tempo2 = pd.read_sql(query_tempo2, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_34944\138465239.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tempo2 = pd.read_sql(query_tempo2, conn)


Estatísticas partidas

In [13]:
df_est_partidas = pd.read_sql(query_est_partidas, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_34944\339199292.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_est_partidas = pd.read_sql(query_est_partidas, conn)


Estatísticas jogadores

In [14]:
df_jogadores = pd.read_sql(query_jogadores, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_34944\922120409.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_jogadores = pd.read_sql(query_jogadores, conn)


Encerrando a conexão

In [15]:
conn.close()

In [16]:
df_jogadores

,data,id_jogo,mando_certo,adversario,campeonato,position,id,temporada,id_jogo,team,...,penalty_conceded,penalty_miss,penalty_save,penalty_won,error_lead_to_a_goal,last_man_tackle,penalty_shootout_miss,penalty_shootout_goal,penalty_shootout_save,cadastrado_em
0,2025-03-29,13473344,Fora,Grêmio,Brasileiro,Goleiro,116,2025,13473344,Atlético,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2025-06-19 12:25:15
1,2025-03-29,13473344,Fora,Grêmio,Brasileiro,Defensor,117,2025,13473344,Atlético,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2025-06-19 12:25:15
2,2025-03-29,13473344,Fora,Grêmio,Brasileiro,Defensor,118,2025,13473344,Atlético,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2025-06-19 12:25:15
3,2025-03-29,13473344,Fora,Grêmio,Brasileiro,Defensor,119,2025,13473344,Atlético,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2025-06-19 12:25:15
4,2025-03-29,13473344,Fora,Grêmio,Brasileiro,Defensor,120,2025,13473344,Atlético,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2025-06-19 12:25:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10125,2025-11-14,14069591,Casa,Atlético Tucumán,Argentino,Meio-campo,3025529,2025,14069591,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-15 09:16:00
10126,2025-11-14,14069591,Casa,Atlético Tucumán,Argentino,Meio-campo,3025530,2025,14069591,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-15 09:16:00
10127,2025-11-14,14069591,Casa,Atlético Tucumán,Argentino,Meio-campo,3025531,2025,14069591,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-15 09:16:00
10128,2025-11-14,14069591,Casa,Atlético Tucumán,Argentino,Atacante,3025532,2025,14069591,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-15 09:16:00


## 1. Métricas de time (coletivas)

### Tratando as informações

Criando uma cópia do DataFrame original

In [17]:
df = df_est_partidas.copy()

Selecionando somente a temporada atual

In [18]:
df = df[df['temporada'] == '2025']

Selecionando somente os últimos 10 jogos

In [19]:
df = df.sort_values('data', ascending=False)

Separando as informações entre o mandante e o visitante

In [20]:
df_casa = df[df['clube_analise'] == casa].head(20)
df_fora = df[df['clube_analise'] == fora].head(20)

Definindo o mandante e o visitante

In [21]:
mandante = df_casa[df_casa['clube'] == casa]

In [22]:
visitante = df_fora[df_fora['clube'] == fora]

### 1.1. Performance geral / resultado

A ideia é mostrar "o quanto o time merece" x "quando o time realmente entrega"

#### xG a favor por jogo

Mandante

In [23]:
xg_casa = df_casa.groupby(['data', 'clube_analise', 'id_jogo', 'clube', 'adversario', 'campeonato'])['expected_goals'].mean().reset_index()

xg_casa_analise = xg_casa[xg_casa['clube'] == casa].sort_values('data', ascending=False)

xg_casa_analise = xg_casa_analise.expected_goals.mean().round(2).astype(float)

xg_casa_analise

0.72

Visitante

In [24]:
xg_fora = df_fora.groupby(['data', 'clube_analise', 'id_jogo', 'clube', 'adversario', 'campeonato'])['expected_goals'].mean().reset_index()

xg_fora_analise = xg_fora[xg_fora['clube'] == fora].sort_values('data', ascending=False)

xg_fora_analise = xg_fora_analise.expected_goals.mean().round(2).astype(float)

xg_fora_analise

1.04

#### xG contra por jogo

Mandante

In [25]:
xg_casa_analise_adversario = xg_casa[(xg_casa['clube_analise'] == casa) & (xg_casa['clube'] != casa)].sort_values('data', ascending=False)

xg_casa_analise_adversario = xg_casa_analise_adversario.expected_goals.mean().round(2).astype(float)

xg_casa_analise_adversario

1.14

Visitante

In [26]:
xg_fora_analise_adversario = xg_fora[(xg_fora['clube_analise'] == fora) & (xg_fora['clube'] !=fora)].sort_values('data', ascending=False)

xg_fora_analise_adversario = xg_fora_analise_adversario.expected_goals.mean().round(2).astype(float)

xg_fora_analise_adversario

0.86

#### Saldo de xG por jogo

Mandante

In [27]:
saldo_xg_casa = (xg_casa_analise - xg_casa_analise_adversario).round(2)

saldo_xg_casa

-0.42

Visitante

In [28]:
saldo_xg_fora = (xg_fora_analise - xg_fora_analise_adversario).round(2)

saldo_xg_fora

0.18

#### Big chances criadas por jogo

Mandante

In [29]:
big_chances = df_casa[['data', 'id_jogo', 'clube_analise', 'clube', 'adversario', 'big_chances']]

big_chances_casa = (big_chances[big_chances['clube'] == casa]['big_chances'].mean()).round(2)

big_chances_casa

1.7

In [30]:
big_chances_casa_adversario = (big_chances[big_chances['clube'] != casa]['big_chances'].mean()).round(2)

big_chances_casa_adversario

1.9

Visitante

In [31]:
big_chances = df_fora[['data', 'id_jogo', 'clube_analise', 'clube', 'adversario', 'big_chances']]

big_chances_fora = big_chances[big_chances['clube'] == fora]['big_chances'].mean().round(2)

big_chances_fora

2.4

In [32]:
big_chances_fora_adversario = big_chances[big_chances['clube'] != fora]['big_chances'].mean().round(2)

big_chances_fora_adversario

1.6

#### Conversão de big chances (%)

Mandante

In [33]:
big_chances_convertidas_casa = df_casa[df_casa['clube'] == casa]['big_chances_scored'].mean()

big_chances_convertidas_casa = round((big_chances_convertidas_casa / big_chances_casa) * 100, 2)

big_chances_convertidas_casa

52.94

In [34]:
big_chances_convertidas_casa_adversario = df_casa[df_casa['clube'] != casa]['big_chances_scored'].mean()

big_chances_convertidas_casa_adversario = round((big_chances_convertidas_casa_adversario / big_chances_casa) * 100, 2)

big_chances_convertidas_casa_adversario

41.18

Visitante

In [35]:
big_chances_convertidas_fora = df_fora[df_fora['clube'] == fora]['big_chances_scored'].mean()

big_chances_convertidas_fora = round((big_chances_convertidas_fora / big_chances_casa) * 100, 2)

big_chances_convertidas_fora

70.59

In [36]:
big_chances_convertidas_fora_adversario = df_fora[df_fora['clube'] != fora]['big_chances_scored'].sum()

big_chances_convertidas_fora_adversario = round((big_chances_convertidas_fora_adversario / big_chances_casa) * 100, 2)

big_chances_convertidas_fora_adversario

352.94

#### Big chances perdidas por jogo

Mandante

In [37]:
big_chances_perdidas_casa = df_casa[df_casa['clube'] == casa]['big_chances_missed'].mean()

big_chances_perdidas_casa

0.8

In [38]:
big_chances_perdidas_casa_adversario = df_casa[df_casa['clube'] != casa]['big_chances_missed'].mean()

big_chances_perdidas_casa_adversario

1.2

Visitante

In [39]:
big_chances_perdidas_fora = df_fora[df_fora['clube'] == fora]['big_chances_missed'].mean()

big_chances_perdidas_fora

1.2

In [40]:
big_chances_perdidas_fora_adversario = df_fora[df_fora['clube'] != fora]['big_chances_missed'].mean()

big_chances_perdidas_fora_adversario

1.0

#### Gols marcados

Mandante

In [41]:
# Selecionando os últimos 10 jogos
df_partida_casa = df_partidas[df_partidas['clube_analise'] == casa].sort_values('data', ascending=False).head(10)

df_partida_gm_casa = df_partida_casa[df_partida_casa['casa'] == casa]['gols_casa'].sum()
df_partida_gm_casa2 = df_partida_casa[df_partida_casa['fora'] == casa]['gols_fora'].sum()

df_partida_gm_casa = round((df_partida_gm_casa + df_partida_gm_casa2) / 10, 2)

df_partida_gm_casa

1.5

Visitante

In [42]:
# Selecionando os últimos 10 jogos
df_partida_fora = df_partidas[df_partidas['clube_analise'] == fora].sort_values('data', ascending=False).head(10)

df_partida_gm_fora = df_partida_fora[df_partida_fora['casa'] == fora]['gols_casa'].sum()
df_partida_gm_fora2 = df_partida_fora[df_partida_fora['fora'] == fora]['gols_fora'].sum()

df_partida_gm_fora = round((df_partida_gm_fora + df_partida_gm_fora2) / 10, 2)

df_partida_gm_fora

1.5

#### Gols sofridos

In [43]:
# Selecionando os últimos 10 jogos
df_partida_casa = df_partidas[df_partidas['clube_analise'] == casa].sort_values('data', ascending=False).head(10)

df_partida_gs_casa = df_partida_casa[df_partida_casa['casa'] == casa]['gols_fora'].sum()
df_partida_gs_casa2 = df_partida_casa[df_partida_casa['fora'] == casa]['gols_casa'].sum()

df_partida_gs_casa = round((df_partida_gs_casa + df_partida_gs_casa2) / 10, 2)

df_partida_gs_casa

1.0

In [44]:
# Selecionando os últimos 10 jogos
df_partida_fora = df_partidas[df_partidas['clube_analise'] == fora].sort_values('data', ascending=False).head(10)

df_partida_gs_fora = df_partida_fora[df_partida_fora['casa'] == fora]['gols_fora'].sum()
df_partida_gs_fora2 = df_partida_fora[df_partida_fora['fora'] == fora]['gols_casa'].sum()

df_partida_gs_fora = round((df_partida_gs_fora + df_partida_gs_fora2) / 10, 2)

df_partida_gs_fora

0.8

#### Over/underperformance xG

Mandante

In [45]:
over_under_xg_casa = round(df_partida_gm_casa - xg_casa_analise, 2)

over_under_xg_casa

0.78

In [46]:
over_under_xg_casa_adversario = round(df_partida_gs_casa - xg_casa_analise_adversario, 2)

over_under_xg_casa_adversario

-0.14

Visitante

In [47]:
over_under_xg_fora = round(df_partida_gm_fora - xg_fora_analise, 2)

over_under_xg_fora

0.46

In [48]:
over_under_xg_fora_adversario = round(df_partida_gs_fora - xg_fora_analise_adversario, 2)

over_under_xg_fora_adversario

-0.06

### 1.2. Ataque: volume, qualidade e perfil

#### 1.2.1 Volume de finalização

##### Chutes por jogo

Mandante

In [49]:
total_chutes_casa = mandante['total_shots'].mean().round(2)

total_chutes_casa

12.6

Visitante

In [50]:
total_chutes_fora = visitante['total_shots'].mean().round(2)

total_chutes_fora

10.0

##### Chutes no alvo / jogo

Mandante

In [51]:
total_chutes_gol_casa = mandante['shots_on_target'].mean().round(2)

total_chutes_gol_casa

4.6

Visitante

In [52]:
total_chutes_gol_fora = visitante['shots_on_target'].mean().round(2)

total_chutes_gol_fora

3.6

##### Chutes fora / jogo

Mandante

In [53]:
total_chutes_fora_casa = mandante['shots_off_target'].mean().round(2)

total_chutes_fora_casa

3.9

Visitante

In [54]:
total_chutes_fora_fora = visitante['shots_off_target'].mean().round(2)

total_chutes_fora_fora

4.3

##### Chutes bloqueados / jogo

Mandante

In [55]:
total_chutes_bloqueados_casa = mandante['blocked_shots'].mean().round(2)

total_chutes_bloqueados_casa

4.1

Visitante

In [56]:
total_chutes_bloqueados_fora = visitante['blocked_shots'].mean().round(2)

total_chutes_bloqueados_fora

2.1

##### Chutes dentro da área (%)

Mandante

In [57]:
chutes = df_casa[df_casa['clube'] == casa]

total_chutes = chutes['total_shots'].sum()
total_chutes_dentro_area = chutes['shots_inside_box'].sum()

total_chutes_dentro_area_casa = round((total_chutes_dentro_area / total_chutes) * 100, 2)

total_chutes_dentro_area_casa

57.94

Visitante

In [58]:
chutes = df_fora[df_fora['clube'] == fora]

total_chutes = chutes['total_shots'].sum()
total_chutes_dentro_area = chutes['shots_inside_box'].sum()

total_chutes_dentro_area_fora = round((total_chutes_dentro_area / total_chutes) * 100, 2)

total_chutes_dentro_area_fora

67.0

##### Chutes fora da área (%)

Mandante

In [59]:
chutes = df_casa[df_casa['clube'] == casa]

total_chutes = chutes['total_shots'].sum()
total_chutes_fora_area = chutes['shots_outside_box'].sum()

total_chutes_fora_area_casa = round((total_chutes_fora_area / total_chutes) * 100, 2)

total_chutes_fora_area_casa

42.06

Visitante

In [60]:
chutes = df_fora[df_fora['clube'] == fora]

total_chutes = chutes['total_shots'].sum()
total_chutes_fora_area = chutes['shots_outside_box'].sum()

total_chutes_fora_area_fora = round((total_chutes_fora_area / total_chutes) * 100, 2)

total_chutes_fora_area_fora

33.0

##### xG por chute

Mandante

In [61]:
chutes = df_casa[df_casa['clube'] == casa]

total_chutes = chutes['total_shots'].sum()
xg_total = chutes['expected_goals'].mean()

xg_total_casa = round((xg_total / total_chutes) * 100, 2)

xg_total_casa

0.57

In [62]:
chutes = df_casa[df_casa['clube'] != casa]

total_chutes = chutes['total_shots'].sum()
xg_total = chutes['expected_goals'].mean()

xg_total_casa_adversario = round((xg_total / total_chutes) * 100, 2)

xg_total_casa_adversario

0.83

Visitante

In [63]:
chutes = df_fora[df_fora['clube'] == fora]

total_chutes = chutes['total_shots'].sum()
xg_total = chutes['expected_goals'].mean()

xg_total_fora = round((xg_total / total_chutes) * 100, 2)

xg_total_fora

1.04

In [64]:
chutes = df_fora[df_fora['clube'] != fora]

total_chutes = chutes['total_shots'].sum()
xg_total = chutes['expected_goals'].mean()

xg_total_fora_adversario = round((xg_total / total_chutes) * 100, 2)

xg_total_fora_adversario

0.49

##### Bolas na trave

Mandante

In [65]:
bolas_trave_casa = df_casa[df_casa['clube'] == casa]['hit_woodwork'].sum()

bolas_trave_casa

4

In [66]:
bolas_trave_casa_adversario = df_casa[df_casa['clube'] != casa]['hit_woodwork'].sum()

bolas_trave_casa_adversario

2

Visitante

In [67]:
bolas_trave_fora = df_fora[df_fora['clube'] == fora]['hit_woodwork'].sum()

bolas_trave_fora

2

In [68]:
bolas_trave_fora_adversario = df_fora[df_fora['clube'] != fora]['hit_woodwork'].sum()

bolas_trave_fora_adversario

5

#### 1.2.2 Criação de chances

##### Passes em profundidade

Mandante

In [69]:
passes_profundidade_casa = mandante['through_balls'].mean().round(2)

passes_profundidade_casa

0.4

Visitante

In [70]:
passes_profundidade_fora = visitante['through_balls'].mean().round(2)

passes_profundidade_fora

0.3

##### Entradas no terço final por jogo

Mandante

In [71]:
entradas_terco_final_casa = mandante['final_third_entries'].mean().round(2)

entradas_terco_final_casa

57.2

In [72]:
entradas_terco_final_casa_adversario = df_casa[df_casa['clube'] != casa]['final_third_entries'].mean()

entradas_terco_final_casa_adversario

56.1

Visitante

In [73]:
entradas_terco_final_fora = visitante['final_third_entries'].mean().round(2)

entradas_terco_final_fora

50.3

In [74]:
entradas_terco_final_fora_adversario = df_fora[df_fora['clube'] != fora]['final_third_entries'].mean()

entradas_terco_final_fora_adversario

69.6

##### Faltas sofridas no terço final / jogo

Mandante

In [75]:
faltas_terco_final_casa = mandante['fouled_in_final_third'].mean().round(2)

faltas_terco_final_casa

2.0

Visitante

In [76]:
faltas_terco_final_fora = visitante['fouled_in_final_third'].mean().round(2)

faltas_terco_final_fora

1.6

##### Escanteios por jogo

Mandante

In [77]:
escanteios_casa = mandante['corner_kicks'].mean().round(2)

escanteios_casa

5.3

In [78]:
mandante_adversario = df_casa[df_casa['clube'] != casa]

escanteios_casa_adversario = mandante_adversario['corner_kicks'].mean().round(2)

escanteios_casa_adversario

4.6

Visitante

In [79]:
escanteios_fora = visitante['corner_kicks'].mean().round(2)

escanteios_fora

2.3

In [80]:
visitante_adversario = df_fora[df_fora['clube'] != fora]

escanteios_fora_adversario = visitante_adversario['corner_kicks'].mean().round(2)

escanteios_fora_adversario

6.2

##### Cruzamentos por jogo

Corrigindo o formato da coluna "crosses"

In [81]:
def tratar_crosses(df, coluna='crosses'):
    # 1) Extrai "acertos" e "totais" da coluna de texto (ex: "5/16 (31%)")
    extraido = df[coluna].str.extract(r'(\d+)/(\d+)').astype(float)

    df['cruzamentos_certos'] = extraido[0].astype(int)
    df['cruzamentos']        = extraido[1].astype(int)

    # 2) Calcula a acurácia em %
    df['acuracia_cruzamentos'] = (
        df['cruzamentos_certos'] / df['cruzamentos'] * 100
    ).round(2)

    # 3) Remove a coluna original
    df.drop(columns=[coluna], inplace=True)

    # retorna o próprio df só por conveniência
    return df

# Aplicando nos dois DataFrames
df_casa = tratar_crosses(df_casa, 'crosses')
df_fora = tratar_crosses(df_fora, 'crosses')

Mandante

In [82]:
cruzamentos_casa = df_casa['acuracia_cruzamentos'].mean().round(2)

cruzamentos_casa

25.97

Visitante

In [83]:
cruzamentos_fora = df_fora['acuracia_cruzamentos'].mean().round(2)

cruzamentos_fora

32.66

##### Cruzamentos por entradas no terço final

Mandante

In [84]:
cruzamentos_ent_terco_final_casa = round((cruzamentos_casa / entradas_terco_final_casa) * 100, 2)

cruzamentos_ent_terco_final_casa

45.4

Visitante

In [85]:
cruzamentos_ent_terco_final_fora = round((cruzamentos_fora / entradas_terco_final_fora) * 100, 2)

cruzamentos_ent_terco_final_fora

64.93

##### Bolas longas por passes

Corrigindo os passes longos

In [86]:
def tratar_long_balls(df, coluna='long_balls'):
    # 1) Extrai "acertos" e "totais" da coluna de texto (ex: "5/16 (31%)")
    extraido = df[coluna].str.extract(r'(\d+)/(\d+)').astype(float)

    df['passes_longos_certos'] = extraido[0].astype(int)
    df['passes_longos']        = extraido[1].astype(int)

    # 2) Calcula a acurácia em %
    df['acuracia_passes_longos'] = (
        df['passes_longos_certos'] / df['passes_longos'] * 100
    ).round(2)

    # 3) Remove a coluna original
    df.drop(columns=[coluna], inplace=True)

    # retorna o próprio df só por conveniência
    return df

# Aplicando nos dois DataFrames
df_casa = tratar_long_balls(df_casa, 'long_balls')
df_fora = tratar_long_balls(df_fora, 'long_balls')

Mandante

In [87]:
total_passes = df_casa['passes'].sum()
total_passes_longos = df_casa['passes_longos'].sum()

passes_longos_por_passes_casa = round((total_passes_longos / total_passes) * 100, 2)

passes_longos_por_passes_casa

11.32

Visitante

In [88]:
total_passes = df_fora['passes'].sum()
total_passes_longos = df_fora['passes_longos'].sum()

passes_longos_por_passes_fora = round((total_passes_longos / total_passes) * 100, 2)

passes_longos_por_passes_fora

14.44

##### Ataque por bola parada

Mandante

In [89]:
ataque_bola_parada_casa = round(escanteios_casa + faltas_terco_final_casa, 0).astype(int)

ataque_bola_parada_casa

7

Visitante

In [90]:
ataque_bola_parada_fora = round(escanteios_fora + faltas_terco_final_fora, 0).astype(int)

ataque_bola_parada_fora

4

### 1.3. Posse e construção

##### Posse média (%)

Ajustando a coluna com a posse de bola

In [91]:
# Mandante
df_casa["ball_possession"] = (
    df_casa["ball_possession"]
      .str.strip()
      .str.replace("%", "", regex=False)
)

# converte para float
df_casa["ball_possession"] = df_casa["ball_possession"].astype(float)


# Visitante
df_fora["ball_possession"] = (
    df_fora["ball_possession"]
      .str.strip()
      .str.replace("%", "", regex=False)
)

# converte para float
df_fora["ball_possession"] = df_fora["ball_possession"].astype(float)

Mandante

In [92]:
posse_bola_casa = df_casa[df_casa['clube'] == casa]['ball_possession'].mean()

posse_bola_casa

49.6

Visitante

In [93]:
posse_bola_fora = df_fora[df_fora['clube'] == fora]['ball_possession'].mean()

posse_bola_fora

45.4

##### Passes por jogo

Mandante

In [94]:
passes_casa = df_casa[df_casa['clube'] == casa]['passes'].mean()

passes_casa

436.7

Visitante

In [95]:
passes_fora = df_fora[df_fora['clube'] == fora]['passes'].mean()

passes_fora

376.5

##### % de acerto de passes

Mandante

In [96]:
passes_certos_casa = df_casa[df_casa['clube'] == casa]['accurate_passes'].mean()

acuracia_passes_casa = round(passes_certos_casa / passes_casa * 100, 2)

acuracia_passes_casa

84.38

Visitante

In [97]:
passes_certos_fora = df_fora[df_fora['clube'] == fora]['accurate_passes'].mean()

acuracia_passes_fora = round(passes_certos_fora / passes_fora * 100, 2)

acuracia_passes_fora

76.36

##### % de bolas longas

Mandante

In [98]:
passes_longos_casa = df_casa[df_casa['clube'] == casa]['passes_longos'].mean()

passes_longos_por_passes = round(passes_longos_casa / passes_casa * 100, 2)

passes_longos_por_passes

11.45

Visitante

In [99]:
passes_longos_fora = df_fora[df_fora['clube'] == fora]['passes_longos'].mean()

acuracia_passes_fora = round(passes_longos_fora / passes_fora * 100, 2)

acuracia_passes_fora

15.35

##### Through balls por 100 passes

Mandante

In [100]:
passes_longos_por_100_passes_casa = round((passes_longos_casa * 100) / passes_casa, 2)

passes_longos_por_100_passes_casa

11.45

Visitante

In [101]:
passes_longos_por_100_passes_fora = round((passes_longos_fora * 100) / passes_fora, 2)

passes_longos_por_100_passes_fora

15.35

### 1.4. Defesa e recuperação

##### Finalizações contra por jogo

Mandante

In [102]:
finalizacoes_casa_adversario = df_casa[df_casa['clube'] != casa]['total_shots'].mean()

finalizacoes_casa_adversario

13.8

Visitante

In [103]:
finalizacoes_fora_adversario = df_fora[df_fora['clube'] != fora]['total_shots'].mean()

finalizacoes_fora_adversario

17.6

##### Bloqueios por jogo

Mandante

In [104]:
bloqueios_casa = df_casa[df_casa['clube'] == casa]['blocked_shots'].mean()

bloqueios_casa

4.1

Visitante

In [105]:
bloqueios_fora = df_fora[df_fora['clube'] == fora]['blocked_shots'].mean()

bloqueios_fora

2.1

##### Desarmes por jogo

Ajustando a coluna

In [106]:
df_casa['tackles_won'] = (
    df_casa['tackles_won']
        .astype(str)
        .str.strip()
        .str.rstrip('%')
        .replace('', pd.NA)
        .astype(float))

df_fora['tackles_won'] = (
    df_fora['tackles_won']
        .astype(str)
        .str.strip()
        .str.rstrip('%')
        .replace('', pd.NA)
        .astype(float))

Mandante

In [107]:
desarmes_casa = df_casa[df_casa['clube'] == casa]['tackles_won'].mean()

desarmes_casa

58.2

Visitante

In [108]:
desarmes_fora = df_fora[df_fora['clube'] == fora]['tackles_won'].mean()

desarmes_fora

64.1

##### % de desarmes vencidos

Acertando a coluna "tackles_won"

In [109]:
df_casa['tackles_won'] = (
    df_casa['tackles_won']
        .astype(str)
        .str.strip()
        .str.rstrip('%')
        .replace('', pd.NA)
        .astype(float))

df_fora['tackles_won'] = (
    df_fora['tackles_won']
        .astype(str)
        .str.strip()
        .str.rstrip('%')
        .replace('', pd.NA)
        .astype(float))

Mandante

In [110]:
desarmes_casa = df_casa[df_casa['clube'] == casa]['tackles_won'].sum()
desarmes_ganhos_casa = df_casa[df_casa['clube'] == casa]['tackles'].sum()

desarmes_ganhos_casa = round(desarmes_ganhos_casa / desarmes_casa * 100, 2)

desarmes_ganhos_casa

25.77

Visitante

In [111]:
desarmes_fora = df_fora[df_fora['clube'] == fora]['tackles_won'].sum()
desarmes_ganhos_fora = df_fora[df_fora['clube'] == fora]['tackles'].sum()

desarmes_ganhos_fora = round(desarmes_ganhos_fora / desarmes_fora * 100, 2)

desarmes_ganhos_fora

26.99

##### Interceptações por jogo

Mandante

In [112]:
interceptacoes_casa = df_casa[df_casa['clube'] == casa]['interceptions'].mean()

interceptacoes_casa

7.4

Visitante

In [113]:
interceptacoes_fora = df_fora[df_fora['clube'] == fora]['interceptions'].mean()

interceptacoes_fora

11.6

##### Recuperações de bola por jogo

Mandante

In [114]:
recuperacoes_casa = df_casa[df_casa['clube'] == casa]['recoveries'].mean()

recuperacoes_casa

41.7

Visitante

In [115]:
recuperacoes_fora = df_fora[df_fora['clube'] == fora]['recoveries'].mean()

recuperacoes_fora

55.1

##### Rebatidas/afastamentos por jogo

Mandante

In [116]:
rebatidas_casa = df_casa[df_casa['clube'] == casa]['clearances'].mean()

rebatidas_casa

24.6

Visitante

In [117]:
rebatidas_fora = df_fora[df_fora['clube'] == fora]['clearances'].mean()

rebatidas_fora

33.9

##### Perdas de posse

Mandante

In [118]:
perda_posse_casa = df_casa[df_casa['clube'] == casa]['dispossessed'].mean()

perda_posse_casa

7.3

Visitante

In [119]:
perda_posse_fora = df_fora[df_fora['clube'] == fora]['dispossessed'].mean()

perda_posse_fora

8.9

#### 1.4.1 Duelo físico

##### Ajustando as colunas

In [120]:
# padrão: won/total (pct%)
PATTERN_DUELS = r'(\d+)\s*/\s*(\d+)\s*\((\d+)%\)'

def process_duels(df, col_original, prefixo):

    print(f"Processando coluna '{col_original}' | existe? ", col_original in df.columns)

    # 1) extrai os números da string
    extraido = (
        df[col_original]
        .astype(str)
        .str.extract(PATTERN_DUELS)
    )

    extraido.columns = ['won_tmp', 'total_tmp', 'pct_tmp']

    # 2) cria colunas numéricas (inteiro com suporte a NaN)
    df[f'{prefixo}_won']   = extraido['won_tmp'].astype('Int64')
    df[f'{prefixo}_total'] = extraido['total_tmp'].astype('Int64')

    # 3) calcula a acurácia em %
    df[f'acuracia_{prefixo}'] = (
        df[f'{prefixo}_won'].astype(float) / df[f'{prefixo}_total'].astype(float) * 100
    ).round(2)

    return df


# --- Depois de RESETAR df_casa e df_fora ---

# Mandante
df_casa = process_duels(df_casa, 'aerial_duels', 'aerial_duels')
df_casa = process_duels(df_casa, 'ground_duels', 'ground_duels')

# Visitante
df_fora = process_duels(df_fora, 'aerial_duels', 'aerial_duels')
df_fora = process_duels(df_fora, 'ground_duels', 'ground_duels')

Processando coluna 'aerial_duels' | existe?  True
Processando coluna 'ground_duels' | existe?  True
Processando coluna 'aerial_duels' | existe?  True
Processando coluna 'ground_duels' | existe?  True


In [121]:
# --- Mandante ---
df_casa = process_duels(df_casa, 'aerial_duels', 'aerial_duels')
df_casa = process_duels(df_casa, 'ground_duels', 'ground_duels')

# --- Visitante ---
df_fora = process_duels(df_fora, 'aerial_duels', 'aerial_duels')
df_fora = process_duels(df_fora, 'ground_duels', 'ground_duels')


Processando coluna 'aerial_duels' | existe?  True
Processando coluna 'ground_duels' | existe?  True
Processando coluna 'aerial_duels' | existe?  True
Processando coluna 'ground_duels' | existe?  True


Ajustando a coluna "duels"

In [122]:
df_casa['duels'] = (
    df_casa['duels']
        .astype(str)
        .str.strip()
        .str.rstrip('%')
        .replace('', pd.NA)
        .astype(float))

df_fora['duels'] = (
    df_fora['duels']
        .astype(str)
        .str.strip()
        .str.rstrip('%')
        .replace('', pd.NA)
        .astype(float))

##### % de duelos vencidos (geral)

Mandante

In [123]:
duelos_ganhos_casa = df_casa[df_casa['clube'] == casa]['duels'].mean()

duelos_ganhos_casa

51.5

Visitante

In [124]:
duelos_ganhos_fora = df_fora[df_fora['clube'] == fora]['duels'].mean()

duelos_ganhos_fora

47.2

##### % de duelos no chão vencidos

Mandante

In [125]:
duelos_chao_ganhos_casa = df_casa[df_casa['clube'] == casa]['acuracia_ground_duels'].mean().round(2)

duelos_chao_ganhos_casa

50.83

Visitante

In [126]:
duelos_chao_ganhos_fora = df_fora[df_fora['clube'] == fora]['acuracia_ground_duels'].mean().round(2)

duelos_chao_ganhos_fora

46.34

##### % de duelos aéreos vencidos

Mandante

In [127]:
duelos_aereos_ganhos_casa = df_casa[df_casa['clube'] == casa]['acuracia_aerial_duels'].mean().round(2)

duelos_aereos_ganhos_casa

52.6

Visitante

In [128]:
duelos_aereos_ganhos_fora = df_fora[df_fora['clube'] == fora]['acuracia_aerial_duels'].mean().round(2)

duelos_aereos_ganhos_fora

49.28

### 1.5. Pressão e intensidade (PPDA “raiz” e proxies)

PPDA – Passes Per Defensive Action (passes permitidos ao adversário por ação defensiva no terço ofensivo/meio). 

Mandante

In [129]:
acoes_defensivas_casa = mandante['tackles'].sum() + mandante['interceptions'].sum() + mandante['recoveries'].sum() + mandante['clearances'].sum()

passes_casa_adversario = mandante = df_casa[df_casa['clube'] != casa]['passes'].sum()

acoes_defensivas_casa = round(passes_casa_adversario / acoes_defensivas_casa, 2)

acoes_defensivas_casa

4.91

Visitante

In [130]:
acoes_defensivas_fora = visitante['tackles'].sum() + visitante['interceptions'].sum() + visitante['recoveries'].sum() + visitante['clearances'].sum()

passes_fora_adversario = visitante = df_fora[df_fora['clube'] != fora]['passes'].sum()

acoes_defensivas_fora = round(passes_fora_adversario / acoes_defensivas_fora, 2)

acoes_defensivas_fora

3.74

### 1.6. Disciplina e erros

#### Cartão amarelo

Mandante

In [131]:
amarelos_casa = df_casa[df_casa['clube'] == casa]['yellow_cards'].sum()

amarelos_casa

22

Visitante

In [132]:
amarelos_fora = df_fora[df_fora['clube'] == fora]['yellow_cards'].sum()

amarelos_fora

20

#### Faltas cometidas por jogo

##### 1. Selecionando os times

Mandante

In [133]:
df_jogadores_casa = df_jogadores[df_jogadores['team'] == casa]

df_jogadores_casa.head(3)

,data,id_jogo,mando_certo,adversario,campeonato,position,id,temporada,id_jogo,team,...,penalty_conceded,penalty_miss,penalty_save,penalty_won,error_lead_to_a_goal,last_man_tackle,penalty_shootout_miss,penalty_shootout_goal,penalty_shootout_save,cadastrado_em
0,2025-03-29,13473344,Fora,Grêmio,Brasileiro,Goleiro,116,2025,13473344,Atlético,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2025-06-19 12:25:15
1,2025-03-29,13473344,Fora,Grêmio,Brasileiro,Defensor,117,2025,13473344,Atlético,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2025-06-19 12:25:15
2,2025-03-29,13473344,Fora,Grêmio,Brasileiro,Defensor,118,2025,13473344,Atlético,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2025-06-19 12:25:15


Visitante

In [134]:
df_jogadores_fora = df_jogadores[df_jogadores['team'] == fora]

df_jogadores_fora.head(3)

,data,id_jogo,mando_certo,adversario,campeonato,position,id,temporada,id_jogo,team,...,penalty_conceded,penalty_miss,penalty_save,penalty_won,error_lead_to_a_goal,last_man_tackle,penalty_shootout_miss,penalty_shootout_goal,penalty_shootout_save,cadastrado_em
265,2025-04-03,13640398,Fora,Puerto Cabello,Sul-Americana,Goleiro,22134,2025,13640398,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-06-19 21:58:51
266,2025-04-03,13640398,Fora,Puerto Cabello,Sul-Americana,Meio-campo,22135,2025,13640398,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-06-19 21:58:51
267,2025-04-03,13640398,Fora,Puerto Cabello,Sul-Americana,Defensor,22136,2025,13640398,Lanús,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2025-06-19 21:58:51


##### 2. Selecionando a temporada 2025 e ordenando as datas da maior para a menor

Mandante

In [135]:
df_jogadores_casa = df_jogadores_casa[df_jogadores_casa['temporada'] == '2025'].sort_values('data', ascending=False)

df_jogadores_casa.head(3)

,data,id_jogo,mando_certo,adversario,campeonato,position,id,temporada,id_jogo,team,...,penalty_conceded,penalty_miss,penalty_save,penalty_won,error_lead_to_a_goal,last_man_tackle,penalty_shootout_miss,penalty_shootout_goal,penalty_shootout_save,cadastrado_em
9557,2025-11-22,14250689,Fora,Lanús,Sul-Americana,Defensor,3034793,2025,14250689,Atlético,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-23 11:55:31
9549,2025-11-22,14250689,Fora,Lanús,Sul-Americana,Defensor,3034785,2025,14250689,Atlético,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-23 11:55:31
9542,2025-11-22,14250689,Fora,Lanús,Sul-Americana,Goleiro,3034778,2025,14250689,Atlético,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2025-11-23 11:55:31


Visitante

In [136]:
df_jogadores_fora = df_jogadores_fora[df_jogadores_fora['temporada'] == '2025'].sort_values('data', ascending=False)

df_jogadores_fora.head(3)

,data,id_jogo,mando_certo,adversario,campeonato,position,id,temporada,id_jogo,team,...,penalty_conceded,penalty_miss,penalty_save,penalty_won,error_lead_to_a_goal,last_man_tackle,penalty_shootout_miss,penalty_shootout_goal,penalty_shootout_save,cadastrado_em
9538,2025-11-22,14250689,Casa,Atlético,Sul-Americana,Atacante,3034768,2025,14250689,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2025-11-23 11:55:31
9532,2025-11-22,14250689,Casa,Atlético,Sul-Americana,Meio-campo,3034762,2025,14250689,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-23 11:55:31
9539,2025-11-22,14250689,Casa,Atlético,Sul-Americana,Defensor,3034769,2025,14250689,Lanús,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-23 11:55:31


##### 3. Selecionando os últimos 10 jogos

Mandante

In [137]:
df_jogadores_casa = df_jogadores_casa.loc[:, ~df_jogadores_casa.columns.duplicated()]

In [138]:
# 1) garantir que 'data' é datetime
df_jogadores_casa['data'] = pd.to_datetime(df_jogadores_casa['data'])

# 2) filtrar só temporada 2025 e ordenar pela data
df_jogadores_casa = (
    df_jogadores_casa[df_jogadores_casa['temporada'] == '2025']
    .sort_values('data', ascending=False)
)

# 3) remover colunas duplicadas (incluindo id_jogo duplicado)
df_jogadores_casa = df_jogadores_casa.loc[:, ~df_jogadores_casa.columns.duplicated()]

# 4) pegar as 10 datas mais recentes (sem repetir)
top10_datas = (
    df_jogadores_casa
        .sort_values('data', ascending=False)['data']
        .drop_duplicates()
        .head(10)
)

# 5) filtrar TODAS as linhas dessas 10 datas
ultimos_10_jogos_casa = (
    df_jogadores_casa[
        df_jogadores_casa['data'].isin(top10_datas)
    ]
    .sort_values('data', ascending=False)   # se quiser, pode adicionar outra coluna aqui
    .reset_index(drop=True)
)

# 6) conferir
ultimos_10_jogos_casa.head(3)

,data,id_jogo,mando_certo,adversario,campeonato,position,id,temporada,team,name,...,penalty_conceded,penalty_miss,penalty_save,penalty_won,error_lead_to_a_goal,last_man_tackle,penalty_shootout_miss,penalty_shootout_goal,penalty_shootout_save,cadastrado_em
0,2025-11-22,14250689,Fora,Lanús,Sul-Americana,Defensor,3034793,2025,Atlético,Renzo Saravia,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-23 11:55:31
1,2025-11-22,14250689,Fora,Lanús,Sul-Americana,Defensor,3034779,2025,Atlético,Ruan,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-23 11:55:31
2,2025-11-22,14250689,Fora,Lanús,Sul-Americana,Meio-campo,3034786,2025,Atlético,Bernard,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-11-23 11:55:31


In [139]:
df_jogadores_casa = ultimos_10_jogos_casa.copy()

Visitante

In [140]:
df_jogadores_fora = df_jogadores_fora.loc[:, ~df_jogadores_fora.columns.duplicated()]

In [141]:
# filtrar temporada e ordenar
df_jogadores_fora = (
    df_jogadores_fora[df_jogadores_fora['temporada'] == '2025']
    .sort_values('data', ascending=False)
)

# remover colunas duplicadas
df_jogadores_fora = df_jogadores_fora.loc[:, ~df_jogadores_fora.columns.duplicated()]

# pegar as 10 datas mais recentes
top10_datas_fora = (
    df_jogadores_fora
        .sort_values('data', ascending=False)['data']
        .drop_duplicates()
        .head(10)
)

# filtrar todas as linhas dessas 10 datas
ultimos_10_jogos_fora = (
    df_jogadores_fora[
        df_jogadores_fora['data'].isin(top10_datas_fora)
    ]
    .sort_values('data', ascending=False)
    .reset_index(drop=True)
)

In [142]:
df_jogadores_fora = ultimos_10_jogos_fora.copy()

##### 4. Retornando as faltas cometidas

Mandante

In [143]:
fatas_cometidas_casa = df_jogadores_casa['fouls'].sum()

fatas_cometidas_casa = round(fatas_cometidas_casa / 10, 2)

fatas_cometidas_casa

10.9

Visitante

In [144]:
fatas_cometidas_fora = df_jogadores_fora['fouls'].sum()

fatas_cometidas_fora = round(fatas_cometidas_fora / 10, 2)

fatas_cometidas_fora

13.1

#### Pênaltis cometidos por jogo

Mandante

In [145]:
penaltis_cometidos_casa = df_jogadores_casa['penalty_conceded'].sum()

penaltis_cometidos_casa = round(penaltis_cometidos_casa / 10, 2)

penaltis_cometidos_casa

0.2

Visitante

In [146]:
penaltis_cometidos_fora = df_jogadores_fora['penalty_conceded'].sum()

penaltis_cometidos_fora = round(penaltis_cometidos_fora / 10, 2)

penaltis_cometidos_fora

0.1

#### Erros que levam à finalização / gol

Mandante

In [147]:
erros_finalizacao = df_jogadores_casa['error_lead_to_a_shot'].sum()
erros_gol = df_jogadores_casa['error_lead_to_a_goal'].sum()

erros_perigosos_casa = round((erros_finalizacao + erros_gol) / 10, 2)

erros_perigosos_casa

0.3

Visitante

In [148]:
erros_finalizacao = df_jogadores_fora['error_lead_to_a_shot'].sum()
erros_gol = df_jogadores_fora['error_lead_to_a_goal'].sum()

erros_perigosos_fora = round((erros_finalizacao + erros_gol) / 10, 2)

erros_perigosos_fora

0.2

## 2. Métricas de jogador (individuais)

### 2.1. Métricas gerais

#### Minutos jogados

Mandante

In [195]:
from IPython.display import display

minutos_jogados_casa = (
    df_jogadores_casa
    .groupby(['player_id', 'name', 'position'])['minutes_played']
    .sum()
    .reset_index()
    .sort_values('minutes_played', ascending=False)
).head(11)

minutos_jogados_casa = minutos_jogados_casa[['name', 'position', 'minutes_played']]

minutos_jogados_casa = display(minutos_jogados_casa.style.hide(axis='index'))

minutos_jogados_casa

name,position,minutes_played
Everson,Goleiro,930
Igor Gomes,Meio-campo,771
Vitor Hugo,Defensor,767
Junior Alonso,Defensor,750
Ruan,Defensor,723
Alan Franco,Meio-campo,715
Hulk,Atacante,674
Guilherme Arana,Defensor,615
Rony,Atacante,602
Dudu,Atacante,592


Visitante

In [197]:
minutos_jogados_fora = (
    df_jogadores_fora
    .groupby(['player_id', 'name', 'position'])['minutes_played']
    .sum()
    .reset_index()
    .sort_values('minutes_played', ascending=False)
).head(11)

minutos_jogados_fora = minutos_jogados_fora[['name', 'position', 'minutes_played']]

minutos_jogados_fora = display(minutos_jogados_fora.style.hide(axis='index'))

minutos_jogados_fora

name,position,minutes_played
Nahuel Losada,Goleiro,930
Gonzalo Perez,Defensor,795
Sasha Marcich,Defensor,772
Carlos Izquierdoz,Defensor,750
José Canale,Defensor,746
Marcelino Moreno,Meio-campo,740
Agustín Medina,Meio-campo,721
Eduardo Salvio,Meio-campo,720
Agustín Cardozo,Meio-campo,717
Rodrigo Castillo,Atacante,675


#### Rating total

Mandante

In [215]:
from IPython.display import display

rating_casa = (
    df_jogadores_casa
    .groupby(['player_id', 'name', 'position'])['rating']
    .sum()
    .reset_index()
)

# arredonda de fato a coluna rating
rating_casa['rating'] = rating_casa['rating'].round(2)

# ordena e pega o top 10
rating_casa = (
    rating_casa
    .sort_values('rating', ascending=False)
    .head(10)[['name', 'position', 'rating']]
)

# formata para exibir sempre com 2 casas decimais e sem índice
rating_casa_styled = rating_casa.style.format({'rating': '{:.2f}'}).hide(axis='index')

rating_casa_styled = display(rating_casa_styled)

rating_casa_styled

name,position,rating
Dudu,Atacante,72.90
Everson,Goleiro,72.20
Hulk,Atacante,71.70
Igor Gomes,Meio-campo,68.30
Rony,Atacante,63.20
Bernard,Meio-campo,63.10
Guilherme Arana,Defensor,62.40
Vitor Hugo,Defensor,61.50
Junior Alonso,Defensor,57.10
Alan Franco,Meio-campo,56.10


Visitante

In [ ]:
rating_fora = (
    df_jogadores_fora
    .groupby(['player_id', 'name', 'position'])['rating']
    .sum()
    .reset_index()
)

# arredonda de verdade
rating_fora['rating'] = rating_fora['rating'].round(2)

# ordena e pega top 10, só com as colunas desejadas
rating_fora = (
    rating_fora
    .sort_values('rating', ascending=False)
    .head(10)[['name', 'position', 'rating']]
)

# exibe sem índice e com 2 casas decimais
rating_fora = display(
    rating_fora
        .style
        .format({'rating': '{:.2f}'})
        .hide(axis='index')
)

# rating_fora continua sendo um DataFrame "normal"
rating_fora

name,position,rating
Nahuel Losada,Goleiro,76.50
Sasha Marcich,Defensor,70.90
Rodrigo Castillo,Atacante,70.70
Marcelino Moreno,Meio-campo,70.10
Eduardo Salvio,Meio-campo,66.00
Agustín Medina,Meio-campo,62.50
Agustín Cardozo,Meio-campo,61.20
Gonzalo Perez,Defensor,60.40
José Canale,Defensor,58.30
Ramiro Carrera,Meio-campo,55.40


#### Rating médio

Mandante

In [217]:
rating_medio_casa = (
    df_jogadores_casa
    .groupby(['player_id', 'name', 'position'])['rating']
    .mean()
    .reset_index()
)

# arredonda de fato a coluna rating
rating_medio_casa['rating'] = rating_medio_casa['rating'].round(2)

# ordena e pega o top 10
rating_medio_casa = (
    rating_medio_casa
    .sort_values('rating', ascending=False)
    .head(10)[['name', 'position', 'rating']]
)

# formata para exibir sempre com 2 casas decimais e sem índice
rating_medio_casa_styled = rating_medio_casa.style.format({'rating': '{:.2f}'}).hide(axis='index')

rating_medio_casa = display(rating_medio_casa_styled)

rating_medio_casa

name,position,rating
Dudu,Atacante,7.29
Everson,Goleiro,7.22
Hulk,Atacante,7.17
Junior Alonso,Defensor,7.14
Rony,Atacante,7.02
Bernard,Meio-campo,7.01
Alan Franco,Meio-campo,7.01
Guilherme Arana,Defensor,6.93
Gustavo Scarpa,Meio-campo,6.91
Renzo Saravia,Defensor,6.84


Visitante

In [219]:
rating_medio_fora = (
    df_jogadores_fora
    .groupby(['player_id', 'name', 'position'])['rating']
    .mean()
    .reset_index()
)

# arredonda de verdade
rating_medio_fora['rating'] = rating_medio_fora['rating'].round(2)

# ordena e pega top 10, só com as colunas desejadas
rating_medio_fora = (
    rating_medio_fora
    .sort_values('rating', ascending=False)
    .head(10)[['name', 'position', 'rating']]
)

# exibe sem índice e com 2 casas decimais
rating_medio_fora = rating_medio_fora = display(
    rating_medio_fora
        .style
        .format({'rating': '{:.2f}'})
        .hide(axis='index')
)

# rating_medio_fora continua sendo um DataFrame "normal"
rating_medio_fora

name,position,rating
Nahuel Losada,Goleiro,7.65
José Canale,Defensor,7.29
Sasha Marcich,Defensor,7.09
Rodrigo Castillo,Atacante,7.07
Marcelino Moreno,Meio-campo,7.01
Agustín Medina,Meio-campo,6.94
Ramiro Carrera,Meio-campo,6.92
Luciano Romero,Defensor,6.90
Carlos Izquierdoz,Defensor,6.89
Nicolás Morgantini,Defensor,6.87


#### Idade média

Convertendo a coluna "date_of_birth_timestamp" para data

In [159]:
# data de hoje (sem horário) – usada igual pros dois DFs
hoje = pd.Timestamp.today().normalize()

def adiciona_idade(df):
    # converte o timestamp em data de nascimento
    df['data_nascimento'] = pd.to_datetime(
        df['date_of_birth_timestamp'],
        unit='s',      # se o timestamp estivesse em ms: unit='ms'
        origin='unix'
    )

    # calcula idade em anos completos
    df['idade'] = ((hoje - df['data_nascimento']).dt.days // 365.25).astype(int)
    return df

# aplica nos dois DataFrames
df_jogadores_casa = adiciona_idade(df_jogadores_casa)
df_jogadores_fora = adiciona_idade(df_jogadores_fora)

Mandante

In [220]:
minutos_jogados_casa = (
    df_jogadores_casa
    .groupby(['player_id', 'name', 'position', 'idade'])['minutes_played']
    .sum()
    .reset_index()
    .sort_values('minutes_played', ascending=False)
).head(11)

media_idade_casa = df_jogadores_casa.idade.mean().round(2)

media_idade_casa

29.25

Visitante

In [166]:
minutos_jogados_fora = (
    df_jogadores_fora
    .groupby(['player_id', 'name', 'position', 'idade'])['minutes_played']
    .sum()
    .reset_index()
    .sort_values('minutes_played', ascending=False)
).head(11)

media_idade_fora = minutos_jogados_fora.idade.mean().round(2)

media_idade_fora

29.0

#### Valor médio de mercado

Mandante

In [167]:
minutos_jogados_casa = (
    df_jogadores_casa
    .groupby(['player_id', 'name', 'position', 'value'])['minutes_played']
    .sum()
    .reset_index()
    .sort_values('minutes_played', ascending=False)
).head(11)

valor_mercado_casa = df_jogadores_casa.value.mean().round(2)

valor_mercado_casa

3488710.69

In [174]:
# formata com separador padrão (3,488,710.69)
tmp = f"{valor_mercado_casa:,.2f}"

# troca vírgula por ponto e ponto por vírgula
tmp = tmp.replace(",", "X").replace(".", ",").replace("X", ".")

# adiciona símbolo do euro
valor_mercado_casa_formatado = f"€ {tmp}"

valor_mercado_casa_formatado

'€ 3.488.710,69'

Visitante

In [175]:
minutos_jogados_fora = (
    df_jogadores_fora
    .groupby(['player_id', 'name', 'position', 'value'])['minutes_played']
    .sum()
    .reset_index()
    .sort_values('minutes_played', ascending=False)
).head(11)

valor_mercado_fora = minutos_jogados_fora.value.mean().round(2)

valor_mercado_fora

1494545.45

In [176]:
# formata com separador padrão (3,488,710.69)
tmp = f"{valor_mercado_fora:,.2f}"

# troca vírgula por ponto e ponto por vírgula
tmp = tmp.replace(",", "X").replace(".", ",").replace("X", ".")

# adiciona símbolo do euro
valor_mercado_fora_formatado = f"€ {tmp}"

valor_mercado_fora_formatado

'€ 1.494.545,45'

#### Toques na bola (média por jogo)

Mandante

In [224]:
toques_bola_casa = (
    df_jogadores_casa
    .groupby(['player_id', 'name', 'position'])['touches']
    .mean()
    .reset_index()
    .sort_values('touches', ascending=False)
).head(10)[['name', 'position', 'touches']]

toques_bola_casa = display(toques_bola_casa.style.format({'touches': '{:.0f}'}).hide(axis='index'))

toques_bola_casa

name,position,touches
Junior Alonso,Defensor,71
Vitor Hugo,Defensor,56
Alan Franco,Meio-campo,54
Igor Gomes,Meio-campo,51
Iván Román,Defensor,50
Guilherme Arana,Defensor,48
Ruan,Defensor,48
Everson,Goleiro,42
Bernard,Meio-campo,37
Renzo Saravia,Defensor,35


Visitante

In [225]:
toques_bola_fora = (
    df_jogadores_fora
    .groupby(['player_id', 'name', 'position'])['touches']
    .mean()
    .reset_index()
    .sort_values('touches', ascending=False)
).head(10)[['name', 'position', 'touches']]

toques_bola_fora = display(toques_bola_fora.style.format({'touches': '{:.0f}'}).hide(axis='index'))

toques_bola_fora

name,position,touches
Carlos Izquierdoz,Defensor,61
Agustín Medina,Meio-campo,57
Gonzalo Perez,Defensor,56
Luciano Romero,Defensor,54
Agustín Cardozo,Meio-campo,51
Ramiro Carrera,Meio-campo,48
Sasha Marcich,Defensor,44
José Canale,Defensor,44
Eduardo Salvio,Meio-campo,44
Marcelino Moreno,Meio-campo,43


#### Passes

Mandante

In [227]:
passes_casa = (
    df_jogadores_casa
    .groupby(['player_id', 'name', 'position'])['total_pass']
    .mean()
    .reset_index()
    .sort_values('total_pass', ascending=False)
).head(10)[['name', 'position', 'total_pass']].round(2)

passes_casa

,name,position,total_pass
4,Junior Alonso,Defensor,58.88
6,Vitor Hugo,Defensor,44.11
20,Iván Román,Defensor,43.67
7,Alan Franco,Meio-campo,42.00
10,Igor Gomes,Meio-campo,40.00
11,Ruan,Defensor,33.88
9,Everson,Goleiro,32.80
18,Guilherme Arana,Defensor,31.78
12,Fausto Vera,Meio-campo,28.33
2,Bernard,Meio-campo,24.78


Visitante

In [229]:
passes_fora = (
    df_jogadores_fora
    .groupby(['player_id', 'name', 'position'])['total_pass']
    .mean()
    .reset_index()
    .sort_values('total_pass', ascending=False)
).head(10)[['name', 'position', 'total_pass']].round(2)

passes_fora

,name,position,total_pass
3,Carlos Izquierdoz,Defensor,45.62
20,Agustín Medina,Meio-campo,42.56
10,Agustín Cardozo,Meio-campo,41.78
16,Gonzalo Perez,Defensor,39.00
22,Luciano Romero,Defensor,30.00
5,Ramiro Carrera,Meio-campo,29.75
11,José Canale,Defensor,29.12
9,Marcelino Moreno,Meio-campo,25.80
2,Eduardo Salvio,Meio-campo,25.20
6,Nahuel Losada,Goleiro,25.10


#### Passes certos (média por jogo)

Mandante

In [230]:
passes_casa = (
    df_jogadores_casa
    .groupby(['player_id', 'name', 'position'])['total_pass']
    .mean()
    .reset_index()
    .sort_values('total_pass', ascending=False)
).head(10)[['name', 'position', 'total_pass']].round(2)

passes_casa

,name,position,total_pass
4,Junior Alonso,Defensor,58.88
6,Vitor Hugo,Defensor,44.11
20,Iván Román,Defensor,43.67
7,Alan Franco,Meio-campo,42.00
10,Igor Gomes,Meio-campo,40.00
11,Ruan,Defensor,33.88
9,Everson,Goleiro,32.80
18,Guilherme Arana,Defensor,31.78
12,Fausto Vera,Meio-campo,28.33
2,Bernard,Meio-campo,24.78


Visitante

In [231]:
passes_fora = (
    df_jogadores_fora
    .groupby(['player_id', 'name', 'position'])['total_pass']
    .mean()
    .reset_index()
    .sort_values('total_pass', ascending=False)
).head(10)[['name', 'position', 'total_pass']].round(2)

passes_fora

,name,position,total_pass
3,Carlos Izquierdoz,Defensor,45.62
20,Agustín Medina,Meio-campo,42.56
10,Agustín Cardozo,Meio-campo,41.78
16,Gonzalo Perez,Defensor,39.00
22,Luciano Romero,Defensor,30.00
5,Ramiro Carrera,Meio-campo,29.75
11,José Canale,Defensor,29.12
9,Marcelino Moreno,Meio-campo,25.80
2,Eduardo Salvio,Meio-campo,25.20
6,Nahuel Losada,Goleiro,25.10
